## ML HW2 手把手教學 

### Logistic regression


In [14]:
import numpy as np
import pandas as pd

We only use one-hot-encoding feature here

In [15]:
def load_data():
    x_train = pd.read_csv('X_train')
    x_test = pd.read_csv('X_test')

    x_train = x_train.values
    x_test = x_test.values

    y_train = pd.read_csv('Y_train', header = None)
    y_train = y_train.values
    y_train = y_train.reshape(-1)


    print(x_train.shape)
    print(x_test.shape)
    return x_train, y_train, x_test

*Use* np.clip to prevent overflow

In [16]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-6, 1-1e-6)

In [17]:
def normalize(x_train, x_test):
    x_all = np.concatenate((x_train, x_test), axis = 0)
    mean = np.mean(x_all, axis = 0)
    std = np.std(x_all, axis = 0)

    index = [0, 1, 3, 4, 5]
    mean_vec = np.zeros(x_all.shape[1])
    std_vec = np.ones(x_all.shape[1])
    mean_vec[index] = mean[index]
    std_vec[index] = std[index]

    x_all_nor = (x_all - mean_vec) / std_vec

    x_train_nor = x_all_nor[0:x_train.shape[0]]
    x_test_nor = x_all_nor[x_train.shape[0]:]

    return x_train_nor, x_test_nor

Gradient descent using adagrad

In [18]:
def train(x_train, y_train):
    b = 0.0
    w = np.zeros(x_train.shape[1])
    lr = 0.001
    epoch = 500
    b_lr = 0
    w_lr = np.ones(x_train.shape[1])
    batch_size = 512
    t = 0

    for e in range(epoch):
        # implementation of gradient descent
        index = np.arange(x_train.shape[0])
        np.random.shuffle(index)
        x_train, y_train = x_train[index], y_train[index]
        
        for i in range(int(x_train.shape[0]/batch_size)):
          t += 1
          x = x_train[i * batch_size: (i+1) * batch_size]
          y = y_train[i * batch_size: (i+1) * batch_size]

          z = np.dot(x, w)+b
          zz = sigmoid(z)
          loss = y - zz

          w_g = (-2) * np.dot(loss, x)
          b_g = (-2) * np.sum(loss)
          w_lr = w_lr + w_g ** 2
          b_lr = b_lr + b_g ** 2

          w = w - lr/np.sqrt(w_lr/t) * w_g
          b = b - lr/np.sqrt(b_lr/t) * b_g
          
          if e%100 == 0:          
            zz[zz >= 0.5] = 1
            zz[zz < 0.5] = 0
            acc = y - zz #shape: (32561,)
            acc[acc == 0] = 2
            acc[acc != 2] = 0
            print('epoch:{}\n Accuracy:{}%\n'.format(t+1, np.sum(acc) * 50 / acc.shape[0]))
    return w, b

新增區段

In [ ]:
x_train, y_train, x_test = load_data()
x_train, x_test = normalize(x_train, x_test)

w, b = train(x_train, y_train)

y_test = np.matmul(x_test, w) + b
y_test[y_test >= 0.5] = 1
y_test[y_test < 0.5] = 0

In [25]:
import csv
with open('predict.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['id','label'])
    for i in range(int(y_test.shape[0])):
      writer.writerow( [i + 1, int(y_test[i])] )

### Tip for math problem
[p1](https://people.eecs.berkeley.edu/~jrs/189/exam/mids14.pdf)  
[p2&3](https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/other-readings/chapter13.pdf)  
[p3](https://stats.stackexchange.com/questions/351549/maximum-likelihood-estimators-multivariate-gaussian)